In [1]:
import transformers
from transformers import pipeline
from tqdm import tqdm

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

In [5]:
# ссылка на модель: https://huggingface.co/cointegrated/rubert-tiny-sentiment-balanced
sentiment_analysis = pipeline("sentiment-analysis", model="cointegrated/rubert-tiny-sentiment-balanced",  return_all_scores=True, truncation=True,
    max_length=512)

Device set to use cpu


In [13]:
sentiment_analysis('а про 39 год из головы выпало? Или тебя так дрессировали ?')

[[{'label': 'negative', 'score': 0.8737419843673706},
  {'label': 'neutral', 'score': 0.12128474563360214},
  {'label': 'positive', 'score': 0.004973289556801319}]]

In [3]:
df = pd.read_json('posts_and_comms.json')

In [11]:
df.iloc[154347]['comm']

' а про 39 год из головы выпало? Или тебя так дрессировали ?'

In [123]:
def sent_probs (text):
    '''возвращает вероятности всех трех классов'''
    sa = sentiment_analysis(text)[0]
    neg = sa[0]['score']
    neut = sa[1]['score']
    pos = sa[2]['score']
    return np.array([neg, neut, pos])

In [143]:
for column in ['comm']:
    negatives, neutrals, positives, sentiments = [],[], [], []
    
    for text in tqdm(df[column]):
        try:
            probs = sent_probs(text)
            negatives.append(probs[0])
            neutrals.append(probs[1])
            positives.append(probs[2])
            sentiments.append(['negative', 'neutral', 'positive'][probs.argmax()])
        except Exception as e:
            print(f"Ошибка при обработке текста '{text}': {str(e)}")
            negatives.append(None)
            neutrals.append(None)
            positives.append(None)
            sentiments.append(None)
    
    df['negative'] = negatives
    df['neutral'] = neutrals
    df['positive'] = positives
    df['sentiment'] = sentiments

100%|████████████████████████████████████████████████████████████████████████| 731252/731252 [3:30:31<00:00, 57.89it/s]


In [145]:
df

,post_id,post,date,comm_id,comm,topic,month,negative,neutral,positive,sentiment
0,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734701,так переезжай в Москву. Что мешает ? Или кто ...,soc_pol,1,0.203689,0.789712,0.006600,neutral
1,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734701,всем россиянам в Москву переезжать? Может над...,soc_pol,1,0.155795,0.838709,0.005496,neutral
2,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734777,Comm: Разберитесь с кидаловом на счетчиках вна...,soc_pol,1,0.009979,0.980549,0.009472,neutral
3,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734777,"а что мешает самому установить счётчик, а не ...",soc_pol,1,0.244068,0.754479,0.001454,neutral
4,44734671,"Чем один вид выплат отличается от другого, кто...",2024-01-31,44734777,"Я не электрик,да и счетчики на воду тоже собс...",soc_pol,1,0.453676,0.526596,0.019728,neutral
...,...,...,...,...,...,...,...,...,...,...,...
731247,46803641,Митингующие в Тбилиси забросали пиротехникой и...,2024-12-01,46804154,"Comm: От терпилы. На раз терпят, значит им нра...",protests,12,0.018463,0.198155,0.783382,positive
731248,46803641,Митингующие в Тбилиси забросали пиротехникой и...,2024-12-01,46804229,Comm: видно грузинам очень хочется жить так же...,protests,12,0.283129,0.642132,0.074739,neutral
731249,46803641,Митингующие в Тбилиси забросали пиротехникой и...,2024-12-01,46805464,Comm: Батька такой мойдан сша моментально разо...,protests,12,0.960569,0.036869,0.002562,negative
731250,46803641,Митингующие в Тбилиси забросали пиротехникой и...,2024-12-01,46806065,Comm: Чачи нахлебались и феерверки пускают)),protests,12,0.056448,0.250551,0.693002,positive


In [153]:
data['sentiment'].value_counts()

sentiment
neutral     398488
negative    216955
positive    115809
Name: count, dtype: int64

In [155]:
def tone_score(df):
    '''добавляет в датафрейм столбец с единым показателем тональности'''
    df['tone_score'] = df['positive'] * 1 + df['neutral'] * 0.5 + df['negative'] * 0
    return df

In [157]:
data = tone_score(data)

In [171]:
data.to_json('sent_an.json', orient='records', force_ascii=False, indent=4)